## A notebook to seamlessly take blast output to GO Slim list

This is a notebook meant to run in a working directory. Please set working directory as variable in next cell

In [1]:
wd="/Users/srlab/Desktop/grace-githubrepos/Practicing-with-Blast/analyses/0918"


In [2]:
cd $wd

/Users/srlab/Desktop/grace-githubrepos/Practicing-with-Blast/analyses/0918


### In this directory you will need three files
1) blastout file in format `-6`    
2) Uniprot GO annotation file (340M) available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted`    
3) GOslim file available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted`

In [4]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0  18.6M      0 --:--:-- --:--:-- --:--:-- 19.1M


In [3]:
!curl -O  http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340M  100  340M    0     0  52.1M      0  0:00:06  0:00:06 --:--:-- 55.9M


In [6]:
#checking if files in directory
!ls 

0915-cgigablast-sprot.tab GO-GOslim.sorted          uniprot-SP-GO.sorted


### Set blastout file as variable

In [7]:
blastout="0915-cgigablast-sprot.tab"


# That should be the last thing you have to Type!

![Run all cell below](https://d1zjcuqflbd5k.cloudfront.net/files/acc_524195/1CsBk?response-content-disposition=inline;%20filename=Screen%20Capture%20on%202017-07-20%20at%2011-50-11.gif&Expires=1500576917&Signature=gyehwKgCTxTHzRrBID1ecQc-aBTBD0si7-XVhQHvbDdboJ2j86KMTwN8XoOZzue7PPY-QT7tIDG4DGTeWXv73VT6zfylPqDz9u19oiGZOyiyyK6EAGlK~8nPzTvwhtRN1uNcqqBuINV94sb2HjK9c6TgsHf1N5eRZF4~4FhIgjY_&Key-Pair-Id=APKAJTEIOJM3LSMN33SA)

In [8]:
!head -2 $blastout

CHOYP_043R.1.5|m.16874	sp|Q06852|SLAP1_CLOTH	56.944	216	61	18	10	197	1388	1599	5.15e-08	55.8
CHOYP_043R.1.5|m.16874	sp|Q06852|SLAP1_CLOTH	58.986	217	60	19	5	197	1572	1783	1.10e-07	54.7


In [9]:
#convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [10]:
!head -2 _blast-sep.tab

CHOYP_043R.1.5	m.16874	sp	Q06852	SLAP1_CLOTH	56.944	216	61	18	10	197	1388	1599	5.15e-08	55.8
CHOYP_043R.1.5	m.16874	sp	Q06852	SLAP1_CLOTH	58.986	217	60	19	5	197	1572	1783	1.10e-07	54.7


In [11]:
#reducing number of columns and sorting 
!awk -v OFS='\t' '{print $4, $1, $14}' < _blast-sep.tab | sort \
> _blast-sort.tab

In [12]:
!head -2 _blast-sort.tab

A0A024SNB7	CHOYP_contig_004745	4.3
A0A098D6U0	CHOYP_LOC594793.1.1	0.98


In [13]:
#joining blast with uniprot annoation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [14]:
!head -2 _blast-annot.tab

A0A098D6U0	CHOYP_LOC594793.1.1	GO:0009058; GO:0016740; GO:0016788; GO:0031177
A0A0B5A7M7	CHOYP_MPI5.1.1	GO:0005179; GO:0005576; GO:0006006


## The following is a script modified from Sam

In [45]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\t\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

Changed the first printf statement to include a tab and re-ran script. Skip to input 45 to see if this works.

In [47]:
!head _blast-GO-unfolded.tab

A0A098D6U0	CHOYP_LOC594793.1.1	GO:0009058
A0A098D6U0	CHOYP_LOC594793.1.1	GO:0016740
A0A098D6U0	CHOYP_LOC594793.1.1	GO:0016788
A0A098D6U0	CHOYP_LOC594793.1.1	GO:0031177
A0A0B5A7M7	CHOYP_MPI5.1.1	GO:0005179
A0A0B5A7M7	CHOYP_MPI5.1.1	GO:0005576
A0A0B5A7M7	CHOYP_MPI5.1.1	GO:0006006
A0A0B5AC98	CHOYP_MPI1.1.1	GO:0005179
A0A0B5AC98	CHOYP_MPI1.1.1	GO:0005576
A0A0B5AC98	CHOYP_MPI1.1.1	GO:0006006


In [48]:
!sort -k3 _blast-GO-unfolded.tab > _blast-GO-unfolded.sorted

In [49]:
!tail _blast-GO-unfolded.sorted

Q9TTS3	CHOYP_ACACA.3.7	GO:2001295
Q9TTS3	CHOYP_ACACA.6.7	GO:2001295
Q9TTS3	CHOYP_ACACA.7.7	GO:2001295
P09917	CHOYP_LOC584481.1.10	GO:2001300
P15428	CHOYP_PGDH.5.5	GO:2001300
C8VQ62	CHOYP_THIKA.1.1	GO:2001307
C8VQ62	CHOYP_contig_020217	GO:2001307
Q4WRY5	CHOYP_contig_040462	GO:2001310
Q8BP40	CHOYP_LOC756773.1.1	GO:2001311
Q9NPH0	CHOYP_contig_031590	GO:2001311


In [50]:
!head GO-GOslim.sorted

GO:0000001	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000002	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000003	reproduction	other biological processes	P
GO:0000006	high affinity zinc uptake transmembrane transporter activity	transporter activity	F
GO:0000007	low-affinity zinc ion transmembrane transporter activity	transporter activity	F
GO:0000009	"alpha-1,6-mannosyltransferase activity"	other molecular function	F
GO:0000010	trans-hexaprenyltranstransferase activity	other molecular function	F
GO:0000011	vacuole inheritance	cell organization and biogenesis	P
GO:0000012	single strand break repair	DNA metabolism	P
GO:0000012	single strand break repair	stress response	P


In [51]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 3 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted | head 

GO:0000001	P38880	CHOYP_contig_028087	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000001	Q00083	CHOYP_CRE_26779.1.2	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000001	Q10070	CHOYP_SCHCODRAFT_79577.1.1	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000001	Q5BF59	CHOYP_contig_044629	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000001	Q5BF59	CHOYP_contig_044709	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000002	A2VDZ3	CHOYP_MEF2A.1.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	A2VDZ3	CHOYP_MEF2C.1.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	P87239	CHOYP_CBEI_4404.1.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	Q3T0L3	CHOYP_PHUM_PHUM229210.1.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	Q5FVR2	CHOYP_BRAFLDRAFT_244533.1.2	mitochondrial genome maintenan

In [52]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 3 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $3, $1, $5, $6}' \
> Blastquery-GOslim.tab

In [53]:
!head Blastquery-GOslim.tab

CHOYP_contig_028087	GO:0000001	cell organization and biogenesis	P
CHOYP_CRE_26779.1.2	GO:0000001	cell organization and biogenesis	P
CHOYP_SCHCODRAFT_79577.1.1	GO:0000001	cell organization and biogenesis	P
CHOYP_contig_044629	GO:0000001	cell organization and biogenesis	P
CHOYP_contig_044709	GO:0000001	cell organization and biogenesis	P
CHOYP_MEF2A.1.1	GO:0000002	cell organization and biogenesis	P
CHOYP_MEF2C.1.1	GO:0000002	cell organization and biogenesis	P
CHOYP_CBEI_4404.1.1	GO:0000002	cell organization and biogenesis	P
CHOYP_PHUM_PHUM229210.1.1	GO:0000002	cell organization and biogenesis	P
CHOYP_BRAFLDRAFT_244533.1.2	GO:0000002	cell organization and biogenesis	P


In [31]:
!join -1 3 -2 1 -t $'\t' _blast-GO-unfolded.sorted GO-GOslim.sorted

In [33]:
!join -1 3 -2 1 -t $'\t' GOunfolded.sorted GOslim.sorted

GO:0000009	Q9TTS3	CHOYP_ACACA.3.7	"alpha-1,6-mannosyltransferase activity"	other molecular function	F
GO:0000010	Q9TTS3	CHOYP_ACACA.6.7	trans-hexaprenyltranstransferase activity	other molecular function	F
GO:0000011	Q9TTS3	CHOYP_ACACA.7.7	vacuole inheritance	cell organization and biogenesis	P
GO:0000012	P09917	CHOYP_LOC584481.1.10	single strand break repair	DNA metabolism	P
GO:0000012	P09917	CHOYP_LOC584481.1.10	single strand break repair	stress response	P
GO:0000012	P15428	CHOYP_PGDH.5.5	single strand break repair	DNA metabolism	P
GO:0000012	P15428	CHOYP_PGDH.5.5	single strand break repair	stress response	P


In [34]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 3 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted

In [35]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 3 -2 1 -t $'\t' -o 2.1 \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted | head 

In [37]:
!cat -t _blast-GO-unfolded.sorted | head

A0JM56^ICHOYP_LOC578913.1.1
A0JNG7^ICHOYP_F16B1.1.1
A0JP70^ICHOYP_LOC100371299.1.1
A0JP70^ICHOYP_contig_000435
A0JPI9^ICHOYP_BRAFLDRAFT_66304.1.3
A0JPI9^ICHOYP_BRAFLDRAFT_87451.2.4
A0JPI9^ICHOYP_LOC100005101.1.1
A0JPI9^ICHOYP_LOC100183078.1.1
A0JPI9^ICHOYP_LOC100372392.4.8
A0JPI9^ICHOYP_LOC100372392.5.8
cat: stdout: Broken pipe


In [38]:
!head _blast-GO-unfolded.sorted

A0JM56	CHOYP_LOC578913.1.1
A0JNG7	CHOYP_F16B1.1.1
A0JP70	CHOYP_LOC100371299.1.1
A0JP70	CHOYP_contig_000435
A0JPI9	CHOYP_BRAFLDRAFT_66304.1.3
A0JPI9	CHOYP_BRAFLDRAFT_87451.2.4
A0JPI9	CHOYP_LOC100005101.1.1
A0JPI9	CHOYP_LOC100183078.1.1
A0JPI9	CHOYP_LOC100372392.4.8
A0JPI9	CHOYP_LOC100372392.5.8


In [39]:
!cat -t _blast-GO-unfolded.sorted | tail

Q9TTS3^ICHOYP_ACACA.3.7^IGO:2001295
Q9TTS3^ICHOYP_ACACA.6.7^IGO:2001295
Q9TTS3^ICHOYP_ACACA.7.7^IGO:2001295
P09917^ICHOYP_LOC584481.1.10^IGO:2001300
P15428^ICHOYP_PGDH.5.5^IGO:2001300
C8VQ62^ICHOYP_THIKA.1.1^IGO:2001307
C8VQ62^ICHOYP_contig_020217^IGO:2001307
Q4WRY5^ICHOYP_contig_040462^IGO:2001310
Q8BP40^ICHOYP_LOC756773.1.1^IGO:2001311
Q9NPH0^ICHOYP_contig_031590^IGO:2001311


In [40]:
!cat -t GOunfolded.sorted

Q9TTS3^ICHOYP_ACACA.3.7^IGO:0000009
Q9TTS3^ICHOYP_ACACA.6.7^IGO:0000010
Q9TTS3^ICHOYP_ACACA.7.7^IGO:0000011
P09917^ICHOYP_LOC584481.1.10^IGO:0000012
P15428^ICHOYP_PGDH.5.5^IGO:0000012
C8VQ62^ICHOYP_THIKA.1.1^IGO:2001307
C8VQ62^ICHOYP_contig_020217^IGO:2001307
Q4WRY5^ICHOYP_contig_040462^IGO:2001310
Q8BP40^ICHOYP_LOC756773.1.1^IGO:2001311
Q9NPH0^ICHOYP_contig_031590^IGO:2001312

In [41]:
!cat -t GOunfolded.sorted

Q9TTS3^ICHOYP_ACACA.3.7^IGO:0000009^I
Q9TTS3^ICHOYP_ACACA.6.7^IGO:0000010^I
Q9TTS3^ICHOYP_ACACA.7.7^IGO:0000011^I
P09917^ICHOYP_LOC584481.1.10^IGO:0000012^I
P15428^ICHOYP_PGDH.5.5^IGO:0000012^I
C8VQ62^ICHOYP_THIKA.1.1^IGO:2001307^I
C8VQ62^ICHOYP_contig_020217^IGO:2001307^I
Q4WRY5^ICHOYP_contig_040462^IGO:2001310^I
Q8BP40^ICHOYP_LOC756773.1.1^IGO:2001311^I
Q9NPH0^ICHOYP_contig_031590^IGO:2001312^I

In [42]:
#possible issues:
#no tab after second column when entry has no GO term
#how does join work with null entries

In [43]:
!cat -t _blast-GO-unfolded.tab | head

A0A098D6U0^ICHOYP_LOC594793.1.1^IGO:0009058
A0A098D6U0^ICHOYP_LOC594793.1.1^IGO:0016740
A0A098D6U0^ICHOYP_LOC594793.1.1^IGO:0016788
A0A098D6U0^ICHOYP_LOC594793.1.1^IGO:0031177
A0A0B5A7M7^ICHOYP_MPI5.1.1^IGO:0005179
A0A0B5A7M7^ICHOYP_MPI5.1.1^IGO:0005576
A0A0B5A7M7^ICHOYP_MPI5.1.1^IGO:0006006
A0A0B5AC98^ICHOYP_MPI1.1.1^IGO:0005179
A0A0B5AC98^ICHOYP_MPI1.1.1^IGO:0005576
A0A0B5AC98^ICHOYP_MPI1.1.1^IGO:0006006
cat: stdout: Broken pipe


In [44]:
!cat -t _blast-GO-unfolded.tab | sort -k3 | head

A0A098D6U0^ICHOYP_LOC594793.1.1^IGO:0009058
A0A098D6U0^ICHOYP_LOC594793.1.1^IGO:0016740
A0A098D6U0^ICHOYP_LOC594793.1.1^IGO:0016788
A0A098D6U0^ICHOYP_LOC594793.1.1^IGO:0031177
A0A0B5A7M7^ICHOYP_MPI5.1.1^IGO:0005179
A0A0B5A7M7^ICHOYP_MPI5.1.1^IGO:0005576
A0A0B5A7M7^ICHOYP_MPI5.1.1^IGO:0006006
A0A0B5AC98^ICHOYP_MPI1.1.1^IGO:0005179
A0A0B5AC98^ICHOYP_MPI1.1.1^IGO:0005576
A0A0B5AC98^ICHOYP_MPI1.1.1^IGO:0006006
sort: write failed: standard output: Broken pipe
sort: write error


In [54]:
!awk '{ print $1 }' GO-GOslim.sorted > GO_only_GOslim.sorted